In [1]:
from bs4 import BeautifulSoup
import pickle

# https://app.roll20.net/campaigns/chatarchive/8981579?p=1&onePage=true&hidewhispers=&hiderollresults=
# save page

# (pagenate https://app.roll20.net/campaigns/chatarchive/8981579?p=1 with cache?)
# annoying data seems to come from graphql responses
with open('data/ChatLog_CoS_1_4_2022.html', mode='r', encoding="utf8") as fd:
    html_doc = fd.read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [2]:
messages = soup.find_all('div', 'message')

print(len(messages))

1143


In [3]:
def is_generic_roll(message_class_list):
    return True if 'rollresult' in message_class_list else False
    
def is_new_character_rolling(message):
    return message.find(class_='spacer')

In [5]:
name_dictionary = {
    'Sasha (Katie)': 'Sasha', 
    'Sasha': 'Sasha', 
    'DM Larissa (GM)': 'NPC', 
    'Larissa J. (GM)': 'NPC', 
    'Ireena': 'NPC', 
    'Cordelia (Steph)': 'Cordelia', 
    'Cordelia Malloyei': 'Cordelia', 
    'Cordelia': 'Cordelia', 
    'Stephanie G.': 'Cordelia', 
    'Thea (Eva)': 'Thea', 
    'Eva Y.' : 'Thea',
    '(To GM)': 'Thea', 
    'Thea': 'Thea', 
    'V (Don)': 'V', 
    'V': 'V', 
    'Don C.': 'V', 
    'Jason W.': 'Ruvi',
    'Ruvi': 'Ruvi'
}

filename = 'data/cos_playerid_dictionary'
infile = open(filename,'rb')
playerid_dictionary = pickle.load(infile)
infile.close()
print(playerid_dictionary)

roll_dictionary = {
    'ATHLETICS' : 'Ability Check',
    'ACROBATICS' : 'Ability Check',
    'SLEIGHT OF HAND' : 'Ability Check',
    'STEALTH' : 'Ability Check',
    'ARCANA' : 'Ability Check',
    'HISTORY' : 'Ability Check',
    'INSIGHT' : 'Ability Check',
    'PERFORMANCE' : 'Ability Check',
    'DECEPTION' : 'Ability Check',
    'PERSUASION' : 'Ability Check',
    'ANIMAL HANDLING' : 'Ability Check',
    'MEDICINE' : 'Ability Check',
    'SURVIVAL' : 'Ability Check',
    'PERCEPTION' : 'Ability Check',
    'INTIMATION' : 'Ability Check',
    'RELIGION' : 'Ability Check',
    'INVESTIGATION' : 'Ability Check',
    'NATURE' : 'Ability Check',
    'STRENGTH' : 'Ability Check',
    'DEXTERITY' : 'Ability Check',
    'CONSTITUTION' : 'Ability Check',
    'INTELLIGENCE' : 'Ability Check',
    'WISDOM' : 'Ability Check',
    'CHARISMA' : 'Ability Check',
    'INITIATIVE' : 'Initiative'  
}

{'player--Mc0wTEzQJUkbCMAS7CL': 'Thea', 'player--Mcr3f4EeiMBacwpwxi_': 'V', 'player--McZsN01dcrQ5YI5j2gv': 'Sasha', 'player--Mc0ymR0LXSEchVVGvpq': 'Cordelia', 'player--MhGfdfTJjEKBhQxs-Xw': 'Ruvi', 'player--Mc0mTv402qre4Jla40P': 'NPC'}


In [6]:
from dateutil.parser import parse

current_time = None
timestamps = []
for message in messages:
    tstamp = message.find(class_='tstamp')
    if tstamp:
        current_time = parse(tstamp.string)
    timestamps.append(current_time)

print(len(timestamps))
print(timestamps[-20:])

1143
[datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 0, 23), datetime.datetime(2021, 11, 1, 1, 19), datetime.datetime(2021, 11, 1, 1, 19), datetime.datetime(2021, 12, 19, 23, 9), datetime.datetime(2021, 12, 19, 23, 9), datetime.datetime(2021, 12, 19, 23, 27), datetime.datetime(2021, 12, 19, 23, 43), datetime.datetime(2021, 12, 19, 23, 43), datetime.datetime(2021, 12, 20, 0, 17), datetime.datetime(2021, 12, 20, 0, 17), datetime.datetime(2021, 12, 20, 0, 17)]


In [7]:
has_roll = []
for message in messages:
    has_roll.append((message.find(class_='inlinerollresult') is not None) or ('rollresult' in message['class']))

print(len([i for i in has_roll if i ]))

roll_msgs = [message for i, message, in zip(has_roll, messages) if i ]

650


In [11]:
import re
characters = [] # Who rolled
roll_types = [] # What kind of roll: (1) Ability Check (2) Saving Throw (3) Initiative (4) Spell/Attack
d20_rolls = [] # Filter rolls made by 1d20 rolls only

roll_re = re.compile(r'">([0-9]+)</span>')

roll_timestamps = []

current_char = ''
    
for message in roll_msgs:    
    is_generic = is_generic_roll(message['class'])
    
    if is_generic:
        # Who
        character_id = ''
        
        for _class in message['class']:
            if 'player--' in _class:
                character_id = _class
        
        character = playerid_dictionary[character_id] # Human-readable conversion
        current_char = character # Update

        # Filter 1d20 only; Amount
        if 'rolling d20' in message.find(class_='formula').get_text():
            amnt = int(message.find(class_='rolled').get_text())
            
            # Append
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
                roll_timestamps.append(current_time)
            else:
                continue
            characters.append(character)
            roll_types.append('Generic')
            d20_rolls.append(amnt)
        
    else:
        character = ''
        
        # Who
        if message.find(class_='you'):
            character = 'Kal'
        elif message.find(class_='spacer'):
            current_char = message.find(class_='by').get_text().strip()[:-1]
            character = name_dictionary[current_char]
        else:
            character = name_dictionary[current_char]
            
        # What type; Amount
        inline_span = message.find('span', class_='inlinerollresult')
        roll_txt = ''
        if 'original-title' in inline_span.attrs:
            roll_txt = inline_span['original-title']
        else:
            roll_txt = inline_span['title']
        if 'Rolling 1d20' in roll_txt:
            amnt = int(roll_re.search(roll_txt).group(1))
            
            sheet_label = message.find(class_='sheet-label')
            if not sheet_label:
                continue
            roll_type_txt = sheet_label.find('span').get_text().split('(')[0].strip().upper()
            roll_type = ''
            
            if roll_type_txt in roll_dictionary:
                roll_type = roll_dictionary[roll_type_txt]
            elif 'SAVE' in roll_type_txt:
                roll_type = 'Saving Throw'
            else:
                roll_type = 'Spell/Attack'
            
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
                roll_timestamps.append(current_time)
            else:
                continue
            characters.append(character)
            roll_types.append(roll_type)
            d20_rolls.append(amnt)
            

print('Characters:', len(characters), characters)
print('Roll Types:', len(roll_types), roll_types)
print('d20 rolls:', len(d20_rolls), d20_rolls)
print('Timestamps:', len(roll_timestamps), roll_timestamps)

Characters: 285 ['Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Cordelia', 'Thea', 'Cordelia', 'Cordelia', 'Cordelia', 'Cordelia', 'Thea', 'Thea', 'Cordelia', 'Thea', 'Cordelia', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Thea', 'Cordelia', 'Sasha', 'Cordelia', 'Sasha', 'Thea', 'Thea', 'Cordelia', 'Sasha', 'Cordelia', 'Thea', 'Cordelia', 'Thea', 'Sasha', 'Cordelia', 'Cordelia', 'V', 'Cordelia', 'Thea', 'Sasha', 'Thea', 'V', 'Sasha', 'Cordelia', 'Cordelia', 'Sasha', 'Thea', 'Sasha', 'Cordelia', 'Thea', 'Cordelia', 'Thea', 'Cordelia', 'Thea', 'Cordelia', 'Thea', 'Sasha', 'Cordelia', 'Sasha', 'Cordelia', 'Thea', 'Cordelia', 'Sasha', 'T

In [30]:
from datetime import timedelta, datetime

MIN_ROLLS = 20
MAX_GAP = timedelta(hours=12)

start_time = roll_timestamps[0]
last_time = roll_timestamps[0]
count = 1

sessions = []

session_idx = []

for time in roll_timestamps:
    session_idx.append(len(sessions))
    print(time, session_idx[-1], count)
    if time - last_time > MAX_GAP:
        print('GAP', count)
        if count > MIN_ROLLS:
            print('COUNT')
            sessions.append((start_time, last_time, count))
            print("added session:", last_time, count)
        count = 1
        start_time = time
    else:
        count += 1
    last_time = time
if count > MIN_ROLLS:
    sessions.append((start_time, last_time, count))
    print(last_time-start_time, count)

print('Sessions:', len(sessions), sessions)
print('Session Idx:', session_idx)
    
with open('data/cos_sessions.txt', 'w') as fd:
    for session in sessions:
        fd.write(session[0].strftime('%Y-%m-%d\n'))

2021-06-12 17:11:00 0 1
2021-06-12 17:12:00 0 2
2021-06-12 18:13:00 0 3
2021-06-12 18:14:00 0 4
2021-06-12 18:14:00 0 5
2021-06-12 18:15:00 0 6
2021-06-12 18:15:00 0 7
2021-06-12 18:16:00 0 8
2021-06-12 18:16:00 0 9
2021-06-12 18:19:00 0 10
2021-06-12 18:26:00 0 11
2021-06-12 18:26:00 0 12
2021-06-12 18:35:00 0 13
2021-06-12 18:35:00 0 14
2021-06-12 18:36:00 0 15
2021-06-12 18:37:00 0 16
2021-06-12 18:38:00 0 17
2021-06-12 18:42:00 0 18
2021-06-12 18:43:00 0 19
2021-06-12 18:43:00 0 20
2021-06-12 18:43:00 0 21
2021-06-12 18:45:00 0 22
2021-06-12 18:47:00 0 23
2021-06-12 18:51:00 0 24
2021-06-12 21:37:00 0 25
2021-06-12 21:38:00 0 26
2021-06-12 21:38:00 0 27
2021-06-12 21:42:00 0 28
2021-06-12 21:51:00 0 29
2021-06-12 22:00:00 0 30
2021-06-12 22:11:00 0 31
2021-06-12 22:16:00 0 32
2021-06-12 22:27:00 0 33
2021-06-12 22:33:00 0 34
2021-06-12 22:36:00 0 35
2021-06-12 22:40:00 0 36
2021-06-13 17:31:00 0 37
GAP 37
COUNT
added session: 2021-06-12 22:40:00 37
2021-06-13 17:31:00 1 1
2021-06-1

In [ ]:
import pandas as pd

roll_df = pd.DataFrame({
    'time': roll_timestamps,
    'character': characters,
    'session': session_idx,
    'type': roll_types,
    'value': d20_rolls
})

roll_df = roll_df.dropna()

roll_df.to_csv('data/rolls.zip', index=False)

print(len(roll_df['character'].unique()))
print(len(roll_df['type'].unique()))
print(len(roll_df['session'].unique()))

print(roll_df['character'].unique())

In [ ]:
roll_df = pd.read_csv('data/rolls.zip', parse_dates=['time'])

print(roll_df.info())

In [ ]:
import plotly.express as px

last_session_df = roll_df[roll_df['time'] > datetime(2021,12,28)]
print(len(last_session_df))

kal_df =  last_session_df[last_session_df['character'] == 'Kal']
print(kal_df)

fig = px.histogram(last_session_df, x="value")
fig.show()

In [ ]:
counts = roll_df.groupby(['character']).count()['time'].sort_values()
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(counts)

In [ ]:
counts = roll_df.groupby(['type']).count()['time'].sort_values()
print(counts)

In [ ]:
data = roll_df.groupby(['session','type']).count()['value'].reset_index(level=[0,1])
print(data)

In [ ]:
print(sum(roll_df['character'] == 'Kal'))